# ShapeNet Dataset explorer

## Plot some samples

In [ ]:
import torch
from torch import nn
from torch import optim
import os.path as osp

import pytorch_lightning as pl
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DataLoader

from pytorch_lightning.loggers import WandbLogger


In [ ]:
category = 'Airplane'  # Pass in `None` to train on all categories.
path = osp.realpath(osp.join('..', 'data', 'ShapeNet'))

pre_transform, transform = T.NormalizeScale(), T.FixedPoints(128)
train_dataset = ShapeNet(path, category, split='train', transform=transform, pre_transform=pre_transform)
valid_dataset = ShapeNet(path, category, split='val', transform=transform, pre_transform=pre_transform)
test_dataset = ShapeNet(path, category, split='test', transform=transform, pre_transform=pre_transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=6)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=6)

In [ ]:
data = train_dataset.get(0)

In [ ]:
from hpcs.utils.viz import plot_cloud

In [ ]:
# data.x contains points features extracted after having applied the transform
# data.pos contains the points coordinates
plotter = plot_cloud(data.x.numpy(), scalars=data.y.numpy(), point_size=3.0, notebook=True)

## Define a Pytorch Lightning module

In [ ]:
class SimpleModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.nn = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )
        
    def forward(self, x):
        y_pred = self.nn(x)
        return y_pred
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch.x, batch.y
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        self.log("train/loss", loss)
        return loss
        

In [ ]:
simpl_mod = SimpleModel()

## Training the model

In [ ]:
wandb_logger = WandbLogger(name='SegTest',save_dir='../logs/')

In [ ]:
trainer = pl.Trainer(limit_train_batches=10, max_epochs=10,logger=wandb_logger)

In [ ]:
trainer.fit(model=simpl_mod, train_dataloaders=train_loader)